In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import gc
import platform
import re
import missingno as msno
%matplotlib inline
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import klib
from matplotlib_venn import venn2

from venn import venn

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)  

color = sns.color_palette('pastel')[:10]
pd.set_option('mode.chained_assignment',  None) # 경고 off

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pickle   
with open('/Users/dli/ABCMART/final_pickle/order_notretail_final.pickle', 'rb') as f:
    order = pickle.load(f)
with open('/Users/dli/ABCMART/final_pickle/user_notretail_final.pickle', 'rb') as f:
    user = pickle.load(f)

In [ ]:
order['is_pos'] = 'order'
order = order.reset_index(drop=True)
a = order.query("카테고리2 == 'SHOE LACE' & 사이즈 =='120끈'")
a["사이즈"] ='120'
order.loc[a.index] = a

In [ ]:
#오프라인(멤버)추출
mem_off_order = order.query("구매처label == 'Offline'")
mem_off_order = mem_off_order.reset_index(drop=True) 

In [ ]:
# load
import pickle
with open('/Users/dli/pos.pickle', 'rb') as f:
    pos = pickle.load(f)

In [ ]:
L = pos.구매처.unique()
L

In [ ]:
# 동일한 구매처에서 비교하기 위해 구매처를 전처리해줍니다.
L = pos.구매처.unique()
mem_off_order=mem_off_order.query('구매처 in @L ')
# 동일한 기간에서 비교하기 위해 기간을 전처리해줍니다.
mem_off_order = mem_off_order.query('구매일자>= "2022-01-01"')
mem_off_order = mem_off_order.reset_index(drop=True) 

In [ ]:
pos.구매일자.min(), pos.구매일자.max()

In [ ]:
mem_off_order.구매일자.min(), mem_off_order.구매일자.max()

In [ ]:
print(len(mem_off_order))
print(len(pos))

# 0. POS데이터 전처리


In [ ]:
pos['구매일자'] = pos['구매일자'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
pos['구매일자']=pos.구매일자.dt.strftime('%Y-%m-%d')
pos['구매시간'] = pos['구매시간'].apply(lambda x: pd.to_datetime(str(x), format='%H%M'))
pos['구매시간']=pos.구매시간.dt.strftime('%H:%M')

In [ ]:
pos['정상가'] = pos['정상가'].apply(lambda x: -x if x<0 else x)
pos['결제가'] = pos['결제가'].apply(lambda x: -x if x<0 else x)
pos

In [ ]:
pos['is_pos'] = 'pos'

In [ ]:
L = pos.columns
for i in L:
    print(i, ':', pos[i].unique())

In [ ]:
# PRODUCTS EXCEPT SHOES -> OTHER PRODUCTS
a = pos.query("카테고리2 == 'PRODUCTS EXCEPT SHOES'")
a["카테고리2"] ='OTHER PRODUCTS'
pos.loc[a.index] = a
#사이즈 전처리
L=['270(XL)', '230(XS)','250(M)' ,'220(XXS)', '260(M)','240(S)','240(XS)','220(S)', '240(M)' ,'260(L)','270(L)','235(SM)' ,'280(XXL)','290(XXXL)' ,'280(XL)' ,'250(S)','230(XXS)']
for i in L : 
    kk = pos[pos['사이즈'] == i]
    kk["사이즈"] = re.sub(r'\([^)]*\)', '',string = i)
    pos.loc[kk.index,:] = kk


In [ ]:
# save
import pickle
with open('pos.pickle', 'wb') as f:
    pickle.dump(pos, f, pickle.HIGHEST_PROTOCOL)

## pos데이터에 order가 있는지 확인

In [ ]:
total=pd.concat([pos[['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','결제가','is_pos']],mem_off_order[['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','결제가','is_pos']]],axis=0)
dupl = total.drop_duplicates(subset=['구매일자','구매처','구매시간','상품명','브랜드명','사이즈'], keep=False)
dupl

In [ ]:
total=pd.concat([pos[['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','결제가','is_pos']],mem_off_order[['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','결제가','is_pos']]],axis=0)
dupl = total.drop_duplicates(subset=['구매일자','구매처','구매시간','상품명','브랜드명','사이즈'], keep=False)
dupl

In [ ]:
# 결제가를 아예 더해줌으로써 pos와 order 맞추기

In [ ]:
pos = pos.reset_index()
pos.rename(columns = {'index':"pos_index"}, inplace=True)

mem_off_order = mem_off_order.reset_index()
mem_off_order.rename(columns = {'index':"order_index"}, inplace=True)

In [ ]:
pos_tmp = pos.groupby(['구매일자','구매시간','브랜드명','카테고리1','상품명','사이즈','구매처','is_pos'], as_index=False)['결제가'].sum()

In [ ]:
order_tmp = pd.DataFrame(mem_off_order.groupby(['브랜드명','상품명','사이즈','구매일자','구매시간','구매처','is_pos','카테고리1'], as_index=False)['결제가'].sum())


In [ ]:
total=pd.concat([pos_tmp ,order_tmp],axis=0)
dupl = total[total.duplicated(subset=['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','카테고리1'],keep=False)]
left_pos = total[~total.duplicated(subset=['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','카테고리1'],keep=False)]
dupl = dupl.reset_index(drop=True)
left_pos = left_pos.reset_index(drop=True)

In [ ]:
for (k1, k2,k3,k4,k5,k6,k7), group in dupl.groupby(['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','카테고리1']):
    if group['결제가'].nunique() ==2 & len(group) >= 2: 
        if dupl.iloc[group.index[0]].is_pos == 'pos':
            dupl.iloc[group.index[1],8] = dupl.iloc[group.index[0]].결제가
        else :  # 0번째 인덱스가 order인경우
            print(group.index)

In [ ]:
new_pos = dupl.query("is_pos =='pos'")
new_order = dupl.query("is_pos =='order'")

In [ ]:
total=pd.concat([new_order,new_pos],axis=0)
#pos와 order에서 ['구매일자','구매처','구매시간','상품명','브랜드명','사이즈']가 겹치는것 맨 처음걸 남기고 나머지는 모두 drop
dupl = total[total.duplicated(subset=['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','결제가','카테고리1'],keep=False)]
dupl = dupl.reset_index(drop=True)

for (k1, k2,k3,k4,k5,k6,k7), group in dupl.groupby(['구매일자','구매처','구매시간','상품명','브랜드명','사이즈','카테고리1']):
    if group['결제가'].nunique() ==2  :
        display(group.head())
        break

In [ ]:
L = list(new_order.상품명.unique())
temp2 = pd.DataFrame(mem_off_order.query("상품명 in @L").groupby("상품명")['카테고리2'].agg([pd.Series.nunique, pd.Series.unique]))
temp2.reset_index(inplace=True)
temp2


In [ ]:
temp2.loc[:,'unique'] = temp2['unique'].apply(lambda x : x[0])
temp2

In [ ]:
new_order = new_order.merge(temp2, how='left', on = '상품명')
new_order

In [ ]:
L = list(left_pos.상품명.unique())
temp = pd.DataFrame(pos.query("상품명 in @L").groupby("상품명")['카테고리2'].agg([pd.Series.nunique, pd.Series.unique]))
temp.reset_index(inplace=True)


In [ ]:
temp.loc[:,'unique'] = temp['unique'].apply(lambda x : x[0])
temp

In [ ]:
left_pos = left_pos.merge(temp, how='left', on = '상품명')

In [ ]:
left_pos.drop(columns = ["nunique"], inplace=True)
left_pos.rename(columns = {'unique':'카테고리2'}, inplace=True)

In [ ]:
new_order.drop(columns = ["nunique"], inplace=True)
new_order.rename(columns = {'unique':'카테고리2'}, inplace=True)

In [ ]:
# save final leftpos
import pickle
with open('left_pos_final.pickle', 'wb') as f:
    pickle.dump(left_pos, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# save final leftpos
import pickle
with open('new_order_final.pickle', 'wb') as f:
    pickle.dump(new_order, f, pickle.HIGHEST_PROTOCOL)

Finally, 겹치는 pos와 order구분함

1. (pos, order 공통) 같은 구매처, 같은시간, 같은 상품 등 겹치는 데이터는 행이 중복으로 찍혀 있어서 결제가를 합침
    - order는 고유번호를 무시하게 되었다. (pos와 비교하기 때문에 개인의 구매행태를 어차피 비교못해서 전처리 편의를 위해 합침)
2. pos 결제가가 order보다 정확하였기 때문에 order와 pos의 중복을 확인한 결과 결제가를 제외한 모든게 같은데 결제가가 다르면 pos의 결제가로 order대체

 
> new_pos : pos에 들어간 회원들의 결제정보   

> new_order : order에 들어간 회원들의 결제정보  

> left_pos : 그 외  

# POS 데이터 요약통계값
- 정상가, 판매가가 1억원이 넘어가는 이상치 발견 후 제거

In [ ]:
#회원 데이터
pos_num = left_pos.describe().T.reset_index()
pos_cate = left_pos[[idx for idx , type_ in zip(left_pos.dtypes.index, left_pos.dtypes) if type_=='object']].describe().T.reset_index()

In [ ]:
#이상치 제거전
pos_num

In [ ]:
#판매가 이상치
sns.boxplot(left_pos['결제가'])

In [ ]:
#order 결제가 기준으로 pos를 맞춰줍니다
new_order.describe().T.reset_index()

In [ ]:
#이상치 제거 후
left_pos = left_pos[left_pos['결제가'] <= new_order['결제가'].max()].reset_index(drop= True)
left_pos.describe().T.reset_index()

In [ ]:
#판매가 이상치 제거
sns.boxplot(left_pos['결제가'])

In [ ]:
#회원 데이터
pos_num = left_pos.describe().T.reset_index()
pos_cate = left_pos[[idx for idx , type_ in zip(left_pos.dtypes.index, left_pos.dtypes) if type_=='object']].describe().T.reset_index()

In [ ]:
# POS 데이터 요약통계값
display(pos_num)
display(pos_cate)

In [ ]:
#회원 데이터
order_num = new_order.describe().T.reset_index()
order_cate = new_order[[idx for idx , type_ in zip(new_order.dtypes.index, new_order.dtypes) if type_=='object']].describe().T.reset_index()

In [ ]:
# order 데이터 요약통계값
display(order_num)
display(order_cate)

In [ ]:
# save final leftpos
import pickle
with open('left_pos_final.pickle', 'wb') as f:
    pickle.dump(left_pos, f, pickle.HIGHEST_PROTOCOL)

# 하루당 평균 판매금액

In [ ]:
# pos 일별평균판매금액
qq = pd.DataFrame(left_pos.groupby("구매일자")['결제가'].sum())
ww = pd.DataFrame(left_pos.reset_index().groupby("구매일자")['index'].nunique())
ww = ww.reset_index().rename(columns={"index": "판매건수"})
tmp=pd.merge(qq,ww, on='구매일자', how='inner')

print("POS 일일평균판매금액은", np.round(qq['결제가'].mean(),0), "원 입니다.")

# 멤버십 일자별 평균판매금액
qq = pd.DataFrame(new_order.groupby("구매일자")['결제가'].sum())
ww = pd.DataFrame(new_order.reset_index().groupby("구매일자")['index'].nunique())
ww = ww.reset_index().rename(columns={"index": "판매건수"})
tmp=pd.merge(qq,ww, on='구매일자', how='inner')
print("멤버십 일일평균판매금액은", np.round(qq['결제가'].mean(),0), "원 입니다.")

# 하루당 평균 판매건수

In [ ]:
#비멤버
qq = pd.DataFrame(left_pos['구매일자'].value_counts())
qq= qq.reset_index().rename(columns={"구매일자": "판매건수"})
print("pos의 일일 평균 판매건수는 " , np.round(qq["판매건수"].mean(),2), "건 입니다." )

#멤버
qq = pd.DataFrame(new_order['구매일자'].value_counts())
qq= qq.reset_index().rename(columns={"구매일자": "판매건수"})
print("멤버십 일일 평균 판매건수는 " , np.round(qq["판매건수"].mean(),2), "건 입니다." )

# 신발 결제주기


- 하루동안 결제주기는?

In [ ]:
left_pos1 = left_pos.query('카테고리1 == "신발"')
qq = pd.DataFrame(left_pos1.groupby('구매일자')['구매시간'].max())
qq = qq.reset_index().rename(columns={"구매시간": "마지막구매시간"})

ww = pd.DataFrame(left_pos1.groupby('구매일자')['구매시간'].min())
ww = ww.reset_index().rename(columns={"구매시간": "최초구매시간"})

total_pos = qq.merge(ww, on ='구매일자',how='left')

In [ ]:
total_pos["timediff"] = pd.to_datetime(total_pos['마지막구매시간'].astype(str)) - pd.to_datetime(total_pos['최초구매시간'].astype(str))
total_pos["timedelta_m"]=total_pos["timediff"].dt.total_seconds()/60
total_pos

In [ ]:
#left_pos
qq = pd.DataFrame(left_pos['구매일자'].value_counts())
qq= qq.reset_index().rename(columns={"구매일자": "판매건수", "index":'구매일자'})
total_pos = total_pos.merge(qq, on ='구매일자', how ='left')
# print("left_pos의 일일 평균 판매건수는 " , np.round(qq["판매건수"].mean(),2), "개 입니다." )

In [ ]:
total_pos['결제주기'] = total_pos['timedelta_m']*60/(total_pos['판매건수']-1) 

In [ ]:
total_pos['결제주기'].mean() 

In [ ]:
sns.histplot(total_pos['결제주기'])

In [ ]:
# off_member
qq = pd.DataFrame(new_order.groupby('구매일자')['구매시간'].max())
qq = qq.reset_index().rename(columns={"구매시간": "마지막구매시간"})

ww = pd.DataFrame(new_order.groupby('구매일자')['구매시간'].min())
ww = ww.reset_index().rename(columns={"구매시간": "최초구매시간"})

total = qq.merge(ww, on ='구매일자',how='left')

In [ ]:
total["timediff"] = pd.to_datetime(total['마지막구매시간'].astype(str)) - pd.to_datetime(total['최초구매시간'].astype(str))
total["timedelta_m"]=total["timediff"].dt.total_seconds()/60
total

In [ ]:
# #멤버
qq = pd.DataFrame(new_order['구매일자'].value_counts())
qq= qq.reset_index().rename(columns={"구매일자": "판매건수", "index":'구매일자'})
total = total.merge(qq, on ='구매일자', how ='left')

In [ ]:
total['결제주기'] = total['timedelta_m']*60/(total['판매건수']-1) #

In [ ]:
total['결제주기'].mean() 

In [ ]:
sns.distplot(total['결제주기'], color='r', label = '멤버')
sns.distplot(total_pos['결제주기'], color = 'b', label='비멤버 + 멤버임을 밝히지 않은 멤버')
plt.title("결제까지 걸리는 평균 시간 (초)")
plt.legend()
plt.show()

# 주요 구매 브랜드
## 1.신발

In [ ]:
#pos
temp = left_pos.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['브랜드명'].value_counts().sort_values(ascending=False)).rename(columns = {'고유번호':"구매건수"})
percent = np.round((total/total['브랜드명'].sum()*100),2)
percent = percent.rename(columns = {'브랜드명':'percent'})
total = total.rename(columns = {'브랜드명':'count'})
cat_pos = pd.concat([total, percent], axis =1, keys=["Total","Percent"])
cat_pos[:10]

In [ ]:
#멤버십
temp = new_order.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['브랜드명'].value_counts().sort_values(ascending=False)).rename(columns = {'고유번호':"구매건수"})
percent = np.round((total/total['브랜드명'].sum()*100),2)
percent = percent.rename(columns = {'브랜드명':'percent'})
total = total.rename(columns = {'브랜드명':'count'})
cat_w = pd.concat([total, percent], axis =1, keys=["Total","Percent"])
cat_w[:10]

In [ ]:
cat_w["Total"][:10].plot(kind = 'bar', color = 'salmon')
plt.title("멤버_신발브랜드별 구매건수", fontsize=20)

cat_pos["Total"][:10].plot(kind = 'bar', color = 'salmon')
plt.title("POS_신발브랜드별 구매건수", fontsize=20)


current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:.0f}'.format(x) for x in current_values])

plt.show()

# 주요구매카테고리 구매비율

In [ ]:
print("카테고리1 별 구매비율 (POS)")
total = pd.DataFrame(left_pos["카테고리1"].value_counts().sort_values(ascending=False)).rename(columns = {'고유번호':"구매건수"})
percent = np.round((total/total['카테고리1'].sum()*100),2)
percent = percent.rename(columns = {'카테고리1':'percent'})
total = total.rename(columns = {'카테고리1':'count'})
cat_w = pd.concat([total, percent], axis =1)
cat_w

In [ ]:
print("카테고리1 별 구매비율 (membership)")
total = pd.DataFrame(new_order["카테고리1"].value_counts().sort_values(ascending=False)).rename(columns = {'고유번호':"구매건수"})
percent = np.round((total/total['카테고리1'].sum()*100),2)
percent = percent.rename(columns = {'카테고리1':'percent'})
total = total.rename(columns = {'카테고리1':'count'})
cat_w = pd.concat([total, percent], axis =1)
cat_w

In [ ]:
print("카테고리2 별 구매비율 (pos)")
total = pd.DataFrame(left_pos["카테고리2"].value_counts().sort_values(ascending=False)).rename(columns = {'고유번호':"구매건수"})
percent = np.round((total/total['카테고리2'].sum()*100),2)
percent = percent.rename(columns = {'카테고리2':'percent'})
total = total.rename(columns = {'카테고리2':'count'})
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

In [ ]:
print("카테고리2 별 구매비율 (멤버십)")
total = pd.DataFrame(new_order["카테고리2"].value_counts().sort_values(ascending=False)).rename(columns = {'고유번호':"구매건수"})
percent = np.round((total/total['카테고리2'].sum()*100),2)
percent = percent.rename(columns = {'카테고리2':'percent'})
total = total.rename(columns = {'카테고리2':'count'})
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

In [ ]:
color = sns.color_palette('pastel')[:10]
a = left_pos["카테고리1"].value_counts()
b= new_order["카테고리1"].value_counts()

plt.figure(figsize = (20, 10))

plt.subplot(1,2,1)
plt.title("카테고리1",fontsize=25)

a.plot(kind = 'pie', subplots=True,autopct='%1.1f%%',textprops= {'fontsize' : 10}, colors=color)

plt.subplot(1,2,2)
plt.title("카테고리1", fontsize=25)

b.plot(kind = 'pie', autopct='%1.1f%%',textprops= {'fontsize' : 10}, colors = color)



plt.suptitle('주요 구매 카테고리1 구매비율 (POS vs 멤버십)', fontsize = 30)
plt.show()

In [ ]:
color = sns.color_palette('pastel')[:10]
a = left_pos["카테고리2"].value_counts()
b= new_order["카테고리2"].value_counts()

plt.figure(figsize = (20, 10))

plt.subplot(1,2,1)
plt.title("카테고리2",fontsize=25)

a.plot(kind = 'pie', subplots=True,autopct='%1.1f%%',textprops= {'fontsize' : 10}, colors=color)

plt.subplot(1,2,2)
plt.title("카테고리2", fontsize=25)

b.plot(kind = 'pie', autopct='%1.1f%%',textprops= {'fontsize' : 10}, colors = color)



plt.suptitle('주요 구매 카테고리2 구매비율 (POS vs 멤버십)', fontsize = 30)
plt.show()

# 신발 카테고리 내 구매 비율
## 신발_카테고리2

In [ ]:
print("신발 카테고리 내 카테고리 2 별 구매비율 (POS)")
temp = left_pos.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['카테고리2'].value_counts().sort_values(ascending=False)).rename(columns = {'카테고리2':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w

In [ ]:
print("신발 카테고리 내 카테고리 2 별 구매비율 (membership)")
temp = new_order.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['카테고리2'].value_counts().sort_values(ascending=False)).rename(columns = {'카테고리2':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w

## 신발_브랜드

In [ ]:
print("신발 카테고리 내 브랜드 별 구매비율 (POS)")
temp = left_pos.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['브랜드명'].value_counts().sort_values(ascending=False)).rename(columns = {'브랜드명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

In [ ]:
print("신발 카테고리 내 브랜드 별 구매비율 (membership)")
temp = new_order.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['브랜드명'].value_counts().sort_values(ascending=False)).rename(columns = {'브랜드명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

## 신발_상품명

In [ ]:
print("신발 카테고리 내 상품 별 구매비율 (POS)")
temp = left_pos.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['상품명'].value_counts().sort_values(ascending=False)).rename(columns = {'상품명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

In [ ]:
print("신발 카테고리 내 상품 별 구매비율 (membership)")
temp = new_order.query("카테고리1 == '신발'")
total = pd.DataFrame(temp['상품명'].value_counts().sort_values(ascending=False)).rename(columns = {'상품명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

# 신발 카테고리 외 구매 비율
## 의류,용품_카테고리2

In [ ]:
print("의류,용품 카테고리 내 카테고리 2 별 구매비율 (POS)")
temp = left_pos.query("카테고리1 != '신발'")
total = pd.DataFrame(temp['카테고리2'].value_counts().sort_values(ascending=False)).rename(columns = {'카테고리2':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w

In [ ]:
print("의류,용품 카테고리 내 카테고리 2 별 구매비율 (membership)")
temp = new_order.query("카테고리1 != '신발'")
total = pd.DataFrame(temp['카테고리2'].value_counts().sort_values(ascending=False)).rename(columns = {'카테고리2':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w

## 의류,용품_브랜드

In [ ]:
print("의류,용품 카테고리 내 브랜드 별 구매비율 (POS)")
temp = left_pos.query("카테고리1 != '신발'")
total = pd.DataFrame(temp['브랜드명'].value_counts().sort_values(ascending=False)).rename(columns = {'브랜드명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

In [ ]:
print("의류,용품 카테고리 내 브랜드 별 구매비율 (membership)")
temp = new_order.query("카테고리1 != '신발'")
total = pd.DataFrame(temp['브랜드명'].value_counts().sort_values(ascending=False)).rename(columns = {'브랜드명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

## 의류,용품_상품명

In [ ]:
print("의류,용품 카테고리 내 상품 별 구매비율 (POS)")
temp = left_pos.query("카테고리1 != '신발'")
total = pd.DataFrame(temp['상품명'].value_counts().sort_values(ascending=False)).rename(columns = {'상품명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

In [ ]:
print("의류,용품 카테고리 내 상품 별 구매비율 (membership)")
temp = new_order.query("카테고리1 != '신발'")
total = pd.DataFrame(temp['상품명'].value_counts().sort_values(ascending=False)).rename(columns = {'상품명':"구매건수"})
percent = np.round((total/total.구매건수.sum()*100),2)
percent.rename(columns= {'구매건수':'percent'}, inplace=True)
# percent.rename(columns= {'구매건수':'percent'}, inplace=True)
cat_w = pd.concat([total, percent], axis =1)
cat_w[:10]

# 구매처

## 1. 총매출액
### 1) POS 구매처 별 총매출액 순위

In [ ]:
print("POS 구매처 별 매출액 순위")
temp = pd.DataFrame(left_pos.groupby("구매처")["결제가"].sum().sort_values(ascending=False)).reset_index()
temp.index = temp.index+1
temp = temp.rename(columns = {'결제가':'POS결제가'})
temp

### 2) 멤버십 구매처 별 총매출액 순위

In [ ]:
print("멤버십 구매처 별 매출액 순위")
temp2 = pd.DataFrame(new_order.groupby("구매처")["결제가"].sum().sort_values(ascending=False)).reset_index()
temp2.index = temp.index+1
temp2 = temp2.rename(columns = {'결제가':'멤버십결제가'})
temp2

In [ ]:
temp = temp.merge(temp2, on='구매처')
temp["총합"]= temp['POS결제가']+temp['멤버십결제가']
temp.loc[:,"POS_결제비율"]=np.round(temp['POS결제가']/(temp['총합']),2)*100
temp.loc[:,"멤버십_결제비율"]=np.round(temp['멤버십결제가']/(temp['총합']),2)*100
temp
temp.drop(columns = ["총합"], inplace=True)

### 3) 구매처 별 POS vs 멤버십 총매출액 비교

In [ ]:
temp.set_index("구매처", inplace=True)
temp

## 2. 평균매출액

### 1) POS 구매처 별 평균매출액 순위

In [ ]:
print("POS 구매처 별 매출액 순위")
temp = pd.DataFrame(left_pos.groupby("구매처")["결제가"].mean().sort_values(ascending=False)).reset_index()
temp.index = temp.index+1
temp = temp.rename(columns = {'결제가':'POS결제가'})
temp

### 2) 멤버십 구매처 별 평균매출액 순위

In [ ]:
print("멤버십 구매처 별 매출액 순위")
temp2 = pd.DataFrame(new_order.groupby("구매처")["결제가"].mean().sort_values(ascending=False)).reset_index()
temp2.index = temp.index+1
temp2 = temp2.rename(columns = {'결제가':'멤버십결제가'})
temp2

In [ ]:
temp = temp.merge(temp2, on='구매처')
temp["총합"]= temp['POS결제가']+temp['멤버십결제가']
temp.loc[:,"POS_결제비율"]=np.round(temp['POS결제가']/(temp['총합']),2)*100
temp.loc[:,"멤버십_결제비율"]=np.round(temp['멤버십결제가']/(temp['총합']),2)*100
temp
temp.drop(columns = ["총합"], inplace=True)

### 3) 구매처 별 POS vs 멤버십 평균매출액 비교

In [ ]:
temp.set_index("구매처", inplace=True)
temp

In [ ]:
left_pos["지점키워드"] = '1'
kk = left_pos[left_pos["구매처"].str.contains("GS")]
kk["지점키워드"] = "GS"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("MS")]
kk["지점키워드"] = "MS"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("ST")]
kk["지점키워드"] = "ST"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("SP")]
kk["지점키워드"] = "SP"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("KM")]
kk["지점키워드"] = "KM"
left_pos.loc[kk.index,:] = kk


kk = left_pos[left_pos["구매처"].str.contains("OTS")]
kk["지점키워드"] = "OTS"
left_pos.loc[kk.index,:] = kk


kk = left_pos[left_pos["구매처"].str.contains("FS")]
kk["지점키워드"] = "FS"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("ES")]
kk["지점키워드"] = "ES"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("DS")]
kk["지점키워드"] = "DS"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("APP")]
kk["지점키워드"] = "APP"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("Mobile")]
kk["지점키워드"] = "Mobile"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("PC")]
kk["지점키워드"] = "PC"
left_pos.loc[kk.index,:] = kk

kk = left_pos[left_pos["구매처"].str.contains("본사특판")]
kk["지점키워드"] = "본사특판"
left_pos.loc[kk.index,:] = kk


In [ ]:
new_order["지점키워드"] = '1'
kk = new_order[new_order["구매처"].str.contains("GS")]
kk["지점키워드"] = "GS"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("MS")]
kk["지점키워드"] = "MS"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("ST")]
kk["지점키워드"] = "ST"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("SP")]
kk["지점키워드"] = "SP"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("KM")]
kk["지점키워드"] = "KM"
new_order.loc[kk.index,:] = kk


kk = new_order[new_order["구매처"].str.contains("OTS")]
kk["지점키워드"] = "OTS"
new_order.loc[kk.index,:] = kk


kk = new_order[new_order["구매처"].str.contains("FS")]
kk["지점키워드"] = "FS"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("ES")]
kk["지점키워드"] = "ES"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("DS")]
kk["지점키워드"] = "DS"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("APP")]
kk["지점키워드"] = "APP"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("Mobile")]
kk["지점키워드"] = "Mobile"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("PC")]
kk["지점키워드"] = "PC"
new_order.loc[kk.index,:] = kk

kk = new_order[new_order["구매처"].str.contains("본사특판")]
kk["지점키워드"] = "본사특판"
new_order.loc[kk.index,:] = kk


In [ ]:
temp = pd.concat([left_pos,new_order])
temp

In [ ]:
def to_percentage(x):
    return str(round(x, 2)) + '%'

In [ ]:
pivot=pd.pivot_table(temp, index="지점키워드", columns ="is_pos", values="결제가", aggfunc="sum", fill_value=0)
pivot=pivot.rename(columns = {"order":"멤버십" , "pos" : "POS"})
for i in pivot.index:
    total = pivot.T[i].sort_values(ascending=False)
    percent = np.round((total/sum(total)*100),3)
    cat_w = pd.concat([total, percent], axis =1, keys=["{}결제 중 멤버십 이용 비율".format(i),"Percent"])
    cat_w['Percent'] = cat_w['Percent'].apply(to_percentage)
    display(cat_w)

In [ ]:
total = temp.groupby(["지점키워드","is_pos"])['결제가'].sum().sort_values(ascending=False)
percent = np.round((total/sum(total)*100),3)
cat_w = pd.concat([total, percent], axis =1, keys=["Total","Percent"])
cat_w['Percent'] = cat_w['Percent'].apply(to_percentage)
cat_w.reset_index(inplace=True)
cat_w.index = cat_w.index+1
display(cat_w[:20])